In [ ]:
#81. RNNによる予測
"""
ID番号で表現された単語列x=(x1,x2,…,xT)
がある．ただし，T
は単語列の長さ，xt∈RV
は単語のID番号のone-hot表記である（V
は単語の総数である）．再帰型ニューラルネットワーク（RNN: Recurrent Neural Network）を用い，単語列x
からカテゴリy
を予測するモデルとして，次式を実装せよ．

h→0=0,h→t=RNN−→−−(emb(xt),h→t−1),y=softmax(W(yh)h→T+b(y))
ただし，emb(x)∈Rdw
は単語埋め込み（単語のone-hot表記から単語ベクトルに変換する関数），h→t∈Rdh
は時刻t
の隠れ状態ベクトル，RNN−→−−(x,h)
は入力x
と前時刻の隠れ状態h
から次状態を計算するRNNユニット，W(yh)∈RL×dh
は隠れ状態ベクトルからカテゴリを予測するための行列，b(y)∈RL
はバイアス項である（dw,dh,L
はそれぞれ，単語埋め込みの次元数，隠れ状態ベクトルの次元数，ラベル数である）．RNNユニットRNN−→−−(x,h)
には様々な構成が考えられるが，典型例として次式が挙げられる．

RNN−→−−(x,h)=g(W(hx)x+W(hh)h+b(h))
ただし，W(hx)∈Rdh×dw，W(hh)∈Rdh×dh,b(h)∈Rdh
はRNNユニットのパラメータ，g
は活性化関数（例えばtanh
やReLUなど）である．

なお，この問題ではパラメータの学習を行わず，ランダムに初期化されたパラメータでy
を計算するだけでよい．次元数などのハイパーパラメータは，dw=300,dh=50
など，適当な値に設定せよ（以降の問題でも同様である）．
"""

import torch
import torch.nn as nn
from torch.utils.data import Dataset
#from knock80 import *

# RNNモデルの定義
#NNクラス：PyTorchのnn.Moduleを継承
#          __init__(初期化メソッド)で隠れ層のサイズ、語彙サイズ、埋め込み層のサイズ、パディングインデックス、および出力サイズを受け取
class RNN(nn.Module):
    def __init__(self, hidden_size, vocab_size, emb_size, pad_idx, output_size):
        super().__init__()
        self.hid_size = hidden_size
        self.emb = nn.Embedding(vocab_size, emb_size, padding_idx=pad_idx)
        self.rnn = nn.RNN(emb_size, hidden_size,
                          nonlinearity="tanh", batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        self.batch_size = x.size()[0]
        hidden = torch.zeros(1, self.batch_size, self.hid_size)
        emb = self.emb(x)
        out, hidden = self.rnn(emb, hidden)
        out = self.fc(out[:, -1, :])
        return out

# データセットの定義
class NewsDataset(Dataset):
    def __init__(self, x, y, tokenizer):
        self.x = x
        self.y = y
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        text = self.x[idx]
        inputs = self.tokenizer(text)

        return {
            'inputs': torch.tensor(inputs, dtype=torch.int64),
            'labels': torch.tensor(self.y[idx], dtype=torch.int64)
        }

# データの読み込み
train = pd.read_csv("train.txt", sep='\t')
valid = pd.read_csv("valid.txt", sep='\t')
test = pd.read_csv("test.txt", sep='\t')

# カテゴリを数値に変換
category = {'b': 0, 't': 1, 'e': 2, 'm': 3}
y_train = torch.tensor(train['CATEGORY'].map(lambda x: category[x]).values)
y_valid = torch.tensor(valid['CATEGORY'].map(lambda x: category[x]).values)
y_test = torch.tensor(test['CATEGORY'].map(lambda x: category[x]).values)

# データセットの作成
dataset_train = NewsDataset(train["TITLE"], y_train, w2id.return_id)
dataset_valid = NewsDataset(valid["TITLE"], y_valid, w2id.return_id)
dataset_test = NewsDataset(test["TITLE"], y_test, w2id.return_id)

if __name__ == "__main__":
    # データセットのサイズを表示
    print(f'len(Dataset): {len(dataset_train)}')
    # データセットの一部を表示
    print('Dataset[index]:')
    for var in dataset_train[1]:
        print(f'  {var}: {dataset_train[1][var]}')

    # ハイパーパラメータの設定
    VOCAB_SIZE = len(set(w2id.id_dict.values())) + 1
    EMB_SIZE = 300
    PADDING_IDX = len(set(w2id.id_dict.values()))
    OUTPUT_SIZE = 4
    HIDDEN_SIZE = 50

    # モデルのインスタンス化
    model = RNN(HIDDEN_SIZE, VOCAB_SIZE, EMB_SIZE, PADDING_IDX, OUTPUT_SIZE)
    # 予測結果を表示
    for i in range(10):
        X = dataset_train[i]['inputs']
        print(torch.softmax(model(X.unsqueeze(0)), dim=-1))
#6. データの読み込みと前処理：読み込んだデータをカテゴリを数値に変換
#                             →NewsDatasetクラスを用いてデータセット作成
train = pd.read_csv("train.txt", sep='\t')
valid = pd.read_csv("valid.txt", sep='\t')
test = pd.read_csv("test.txt", sep='\t')

category = {'b': 0, 't': 1, 'e': 2, 'm': 3}
y_train = torch.tensor(train['CATEGORY'].map(lambda x: category[x]).values)
y_valid = torch.tensor(valid['CATEGORY'].map(lambda x: category[x]).values)
y_test = torch.tensor(test['CATEGORY'].map(lambda x: category[x]).values)

dataset_train = NewsDataset(train["TITLE"], y_train, w2id.return_id)
dataset_valid = NewsDataset(valid["TITLE"], y_valid, w2id.return_id)
dataset_test = NewsDataset(test["TITLE"], y_test, w2id.return_id)

if __name__ == "__main__":
    print(f'len(Dataset): {len(dataset_train)}')
    print('Dataset[index]:')
    for var in dataset_train[1]:
        print(f'  {var}: {dataset_train[1][var]}')

    VOCAB_SIZE = len(set(w2id.id_dict.values())) + 1
    EMB_SIZE = 300
    PADDING_IDX = len(set(w2id.id_dict.values()))
    OUTPUT_SIZE = 4
    HIDDEN_SIZE = 50

    model = RNN(HIDDEN_SIZE, VOCAB_SIZE, EMB_SIZE, PADDING_IDX, OUTPUT_SIZE)
    for i in range(10):
        X = dataset_train[i]['inputs']
        print(torch.softmax(model(X.unsqueeze(0)), dim=-1))

len(Dataset): 10672
Dataset[index]:
  inputs: tensor([2630, 1875,    0, 2630, 1875,   12, 2631,    0,    0,  212,   55,   25,
         657])
  labels: 2


<ipython-input-3-a9ea6c48951d>:76: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'labels': torch.tensor(self.y[idx], dtype=torch.int64)


tensor([[0.1764, 0.1746, 0.4037, 0.2453]], grad_fn=<SoftmaxBackward0>)
tensor([[0.2021, 0.2516, 0.3452, 0.2010]], grad_fn=<SoftmaxBackward0>)
tensor([[0.1890, 0.1602, 0.3613, 0.2896]], grad_fn=<SoftmaxBackward0>)
tensor([[0.2091, 0.3281, 0.2623, 0.2005]], grad_fn=<SoftmaxBackward0>)
tensor([[0.2502, 0.1494, 0.2585, 0.3420]], grad_fn=<SoftmaxBackward0>)
tensor([[0.1823, 0.2169, 0.2799, 0.3208]], grad_fn=<SoftmaxBackward0>)
tensor([[0.2161, 0.3351, 0.2552, 0.1936]], grad_fn=<SoftmaxBackward0>)
tensor([[0.1310, 0.1538, 0.3206, 0.3946]], grad_fn=<SoftmaxBackward0>)
tensor([[0.1748, 0.3143, 0.2383, 0.2727]], grad_fn=<SoftmaxBackward0>)
tensor([[0.1486, 0.3140, 0.3293, 0.2081]], grad_fn=<SoftmaxBackward0>)
len(Dataset): 10672
Dataset[index]:
  inputs: tensor([2630, 1875,    0, 2630, 1875,   12, 2631,    0,    0,  212,   55,   25,
         657])
  labels: 2
tensor([[0.4026, 0.1697, 0.1654, 0.2624]], grad_fn=<SoftmaxBackward0>)
tensor([[0.3675, 0.1478, 0.3130, 0.1717]], grad_fn=<SoftmaxBackwar

<ipython-input-3-a9ea6c48951d>:76: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'labels': torch.tensor(self.y[idx], dtype=torch.int64)
